In [4]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_aws import  ChatBedrock

model_id = os.environ.get("MODEL_ID", "us.anthropic.claude-3-5-haiku-20241022-v1:0")
llm =ChatBedrock(model = model_id,  beta_use_converse_api=True, model_kwargs={"temperature": 0})
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print("dialect:",db.dialect)
print ("tables:")
print(db.get_usable_table_names())
agent_executor = create_sql_agent(llm, db=db, verbose=True)

def query_db(consulta):   
    response = agent_executor.invoke(consulta)
    return response.get("output")


dialect: sqlite
tables:
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [40]:
for t in  agent_executor.tools:
    print (f"{t.name}:")
    print(t.description, end="\n\n")

sql_db_query:
Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema:
Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables:
Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker:
Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [2]:
query = "que tablas conoces?"
response = query_db(query)
print (response)

dialect: sqlite
tables:
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


> Entering new SQL Agent Executor chain...
Thought: I'll use the sql_db_list_tables tool to see what tables are available in the database.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: I see a list of tables that appear to be from a music-related database. The tables include Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, and Track. This looks like a comprehensive database for a music store or streaming service.

Final Answer: Las tablas que conozco son: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, y Track. Parece ser una base de datos relacionada con música, posiblemente de una tienda de música o servicio de streaming.


In [ ]:
import boto3
client = boto3.client('bedrock-agent-runtime')
agent_id = "YOURAGENTID"
agent_alias_id = "TSTALIASID"
input_text = "Hola cuantos discos se han vendido por país?"

In [10]:

import boto3
from botocore.exceptions import ClientError
import logging
logger = logging.getLogger(__name__)


class BedrockAgentService:
    def __init__(self, agent_id, alias_id= "TSTALIASID", client=None) -> None:
        self.agents_runtime_client = client if client else boto3.client('bedrock-agent-runtime')
        self.agent_id = agent_id
        self.alias_id = alias_id

    # from https://docs.aws.amazon.com/code-library/latest/ug/python_3_bedrock-agent-runtime_code_examples.html
    def invoke_agent(self, session_id, prompt):

        try:

            response = self.agents_runtime_client.invoke_agent(
                agentId=self.agent_id,
                agentAliasId=self.alias_id,
                sessionId=session_id,
                inputText=prompt,
            )

            completion = ""

            for event in response.get("completion"):
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

        except ClientError as e:
            logger.error(f"Couldn't invoke agent. {e}")
            raise

        return completion



In [7]:
agent = BedrockAgentService(agent_id, agent_alias_id)

In [ ]:
agent.invoke_agent('12345678', "cuantos discos vendidos por país?")

'Según los datos, los 3 principales países por número de álbumes vendidos son:\n1. Estados Unidos: 205 álbumes\n2. Canadá: 136 álbumes\n3. Francia: 99 álbumes\n\nEn total, se han identificado ventas de álbumes en al menos 10 países diferentes, con Estados Unidos liderando significativamente las ventas.'

In [ ]:
agent.invoke_agent('12345678', "y canciones?")

'Los 3 principales países por número de canciones vendidas son:\n1. Estados Unidos: 494 canciones\n2. Canadá: 304 canciones\n3. Francia y Brasil (empatados): 190 canciones\n\nLa distribución de ventas de canciones sigue un patrón similar a la de álbumes, con Estados Unidos dominando claramente las ventas, seguido por Canadá y luego Francia y Brasil.'

In [ ]:
agent.invoke_agent('12345678', "ahora revenue")

'Los 3 principales países por ingresos totales son:\n1. Estados Unidos: $523.06\n2. Canadá: $303.96\n3. Francia: $195.10\n\nLa distribución de ingresos es consistente con las ventas de álbumes y canciones, con Estados Unidos liderando claramente, seguido por Canadá y Francia. Hay una variación significativa en los ingresos entre los diferentes países, reflejando las diferencias en el volumen de ventas.'